In [8]:
import time
from torchvision import transforms, datasets
import torch
from torch.utils.data import Subset
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.init as init
import random

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])

#how to get train and test data
train_data = datasets.FashionMNIST('path', download=True, train=True, transform=transform)
test_data = datasets.FashionMNIST('path', download=True, train=False, transform=transform)


train_indices = list(range(len(train_data)))
random.shuffle(train_indices)
train_indices_10_percent = train_indices[:len(train_indices)//10]

test_indices = list(range(len(test_data)))
random.shuffle(test_indices)
test_indices_10_percent = test_indices[:len(test_indices)//10]

train_data_10_percent = Subset(train_data, train_indices_10_percent)
test_data_10_percent = Subset(test_data, test_indices_10_percent)

#Define a dataloader to load data
train_loader = torch.utils.data.DataLoader(train_data_10_percent, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data_10_percent, batch_size=16, shuffle=True)

print(len(train_loader))
print(len(test_loader))    
def train(model, criterion, optimizer, data_loader, test_loader, epochs):
    strart_timestamp = time.time()
    training_loss = []
    test_loss = []
    for epoch in range(epochs):
        print(epoch)
        for images, labels in data_loader:
            # Flatten images
            images = images.view(images.shape[0], -1)
            # Zero out the gradients
            optimizer.zero_grad()
            # Forward pass, get our logits
            logits = model(images)
            # Calculate the loss with the logits and the labels
            loss = criterion(logits, labels)
            # Backward pass
            loss.backward()
            # Update the weights
            optimizer.step()

        training_loss.append(loss.item())
        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")
        with torch.no_grad():
            for images, labels in test_loader:
                # Flatten images
                images = images.view(images.shape[0], -1)
                # Forward pass, get our logits
                logits = model(images)
                # Calculate the loss with the logits and the labels
                loss = criterion(logits, labels)
            test_loss.append(loss.item())

    print(f"\nTraining Time (in minutes) = {(time.time()-strart_timestamp)/60:.2f}")
    print(training_loss, test_loss)
    return training_loss, test_loss

def plot_loss(losses, title):
    train, test = losses
    plt.plot(train)
    plt.plot(test)
    plt.legend(['Train', 'Test'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.ylim(0, 3)
    plt.show()

# Define the network architecture
model = nn.Sequential(nn.Linear(784, 128),
                      nn.LeakyReLU(),
                      nn.Linear(128, 64),
                      nn.LeakyReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim = 1))

def init_xavier(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)

model.apply(init_xavier)

# Define the loss
criterion = nn.NLLLoss();

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.003)
epochs = 40



losses = train(model, criterion, optimizer, train_loader, test_loader, epochs=epochs)
plot_loss(losses, 'Losses')



375
63
0
Epoch 1/40 - Loss: 0.5244547724723816
1
Epoch 2/40 - Loss: 0.5824915170669556
2
Epoch 3/40 - Loss: 0.7102937698364258
3
Epoch 4/40 - Loss: 0.2919381260871887
4
Epoch 5/40 - Loss: 0.23958824574947357
5
Epoch 6/40 - Loss: 0.3880336284637451
6
